<a href="https://colab.research.google.com/github/junior8801/PySpark/blob/main/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Utilizando Spark

Neste notebook será utilizado o PySpark para execução de comandos SQL em uma base de dados.

In [4]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=1c02e068dbbb9e59055f8c20af8f66308a5de614ad2fada190ffacd7c031a872
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [1]:
!pip install pyspark


     |████████████████████████████████| 212.4 MB 65 kB/s 
     |████████████████████████████████| 198 kB 67.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=99296d0a274c68a275a64cd3e0fdad70430e109328f08c8b9fcf43ac71ec7f34
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
## iniciar sessão
from pyspark.sql import *
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
## importar funções
from pyspark.sql.functions import *

In [8]:
import pandas as pd
import wget
from zipfile import ZipFile 

In [9]:
## Baixar arquivos

url_base = "http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/"
arquivos_zip = []
for ano in range(2011,2021):
  arquivos_zip.append(f'itr_cia_aberta_{ano}.zip')
for arq in arquivos_zip:
  wget.download(url_base+arq)

## Descompactar
for arq in arquivos_zip:
  ZipFile(arq, 'r').extractall('CVM')

## Agrupar arquivos
nomes = ['DRE_con']
for nome in nomes:
  arquivo = pd.DataFrame()
  for ano in range(2011,2020):
    arquivo = pd.concat([arquivo, pd.read_csv(f'CVM/itr_cia_aberta_{nome}_{ano}.csv',sep=';',decimal=',',encoding='ISO-8859-1')])
  arquivo.to_csv(f'itr_cia_aberta_{nome}_2011_2020',index=False)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.01,Receita de Venda de Bens e/ou Serviços,6159172.0,S
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.01,Receita de Venda de Bens e/ou Serviços,8551982.0,S
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-960060.0,S
3,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-1127021.0,S
4,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.02.01,Encargos Setoriais,-340660.0,N


In [13]:
df = spark.read.csv('/content/itr_cia_aberta_DRE_con_2011_2020',header=True,sep=",",inferSchema=True)
df.show()

+------------------+----------+------+--------------------+------+--------------------+-----+------------+-----------+------------+------------+----------+--------------------+----------+-------------+
|          CNPJ_CIA|  DT_REFER|VERSAO|           DENOM_CIA|CD_CVM|           GRUPO_DFP|MOEDA|ESCALA_MOEDA|ORDEM_EXERC|DT_INI_EXERC|DT_FIM_EXERC|  CD_CONTA|            DS_CONTA|  VL_CONTA|ST_CONTA_FIXA|
+------------------+----------+------+--------------------+------+--------------------+-----+------------+-----------+------------+------------+----------+--------------------+----------+-------------+
|00.001.180/0001-26|2011-03-31|     1|CENTRAIS ELET BRA...|  2437|DF Consolidado - ...| REAL|         MIL|  PENÚLTIMO|  2010-01-01|  2010-03-31|      3.01|Receita de Venda ...| 6159172.0|            S|
|00.001.180/0001-26|2011-03-31|     1|CENTRAIS ELET BRA...|  2437|DF Consolidado - ...| REAL|         MIL|     ÚLTIMO|  2011-01-01|  2011-03-31|      3.01|Receita de Venda ...| 8551982.0|     

In [66]:
#df.describe().show()
df.select('DS_CONTA').dropDuplicates().head(10)

[Row(DS_CONTA='Fornecimento de Energia Elétrica'),
 Row(DS_CONTA='Outras despesas administrativas'),
 Row(DS_CONTA='Encargos de transmissão'),
 Row(DS_CONTA='Juros/Custos Debêntures'),
 Row(DS_CONTA='Depreciação Amortização'),
 Row(DS_CONTA='Variação da Provisão de  Sinistros Ocorridos mas não Avisados'),
 Row(DS_CONTA='Planos previdenciário e assistencial'),
 Row(DS_CONTA='Operações de crédito'),
 Row(DS_CONTA='Custo do fornecimento de energia eletrica'),
 Row(DS_CONTA='Energia Elétrica Comprada para Revenda - PROINFA')]

In [65]:
df.createOrReplaceTempView("DF")
spark.sql("select * FROM DF where CNPJ_CIA='07.628.528/0001-59' AND ORDEM_EXERC <> 'ÚLTIMO' and DS_CONTA='Resultado Financeiro' and CAST( months_between( DT_FIM_EXERC, DT_INI_EXERC ) AS INT )=2 order by CNPJ_CIA,DT_FIM_EXERC").show()


+------------------+----------+------+--------------------+------+--------------------+-----+------------+-----------+------------+------------+--------+--------------------+--------+-------------+
|          CNPJ_CIA|  DT_REFER|VERSAO|           DENOM_CIA|CD_CVM|           GRUPO_DFP|MOEDA|ESCALA_MOEDA|ORDEM_EXERC|DT_INI_EXERC|DT_FIM_EXERC|CD_CONTA|            DS_CONTA|VL_CONTA|ST_CONTA_FIXA|
+------------------+----------+------+--------------------+------+--------------------+-----+------------+-----------+------------+------------+--------+--------------------+--------+-------------+
|07.628.528/0001-59|2011-09-30|     2|BRASILAGRO - CIA ...| 20036|DF Consolidado - ...| REAL|         MIL|  PENÚLTIMO|  2010-07-01|  2010-09-30|    3.06|Resultado Financeiro|  2592.0|            S|
|07.628.528/0001-59|2011-12-31|     2|BRASILAGRO - CIA ...| 20036|DF Consolidado - ...| REAL|         MIL|  PENÚLTIMO|  2010-10-01|  2010-12-31|    3.06|Resultado Financeiro|   541.0|            S|
|07.628.52